# Quantum Circuit Simulator

In [1]:
import numpy as np
from numpy.random import choice
from scipy.optimize import minimize

In [27]:
class Quantum_Circuit:
    def __init__ (self, total_qubit):
        self.total_qubit = total_qubit
        if self.total_qubit <= 0:
            raise Exception ("Please enter valid number greater than 0")
        self.state = {}
        for i in range(total_qubit):
            self.state[i] = np.array([1,0])
            
    def set_initial_state(self, initial_state, target_qubit):
        '''
        initial_state: set any initial state for a target qubit 
        if no initial state is defined then returns the ground state
        '''
        if target_qubit >= self.total_qubit:
            raise Exception("Invalid target_qubit")
        if int(round((np.abs(initial_state)**2).sum(),5)) != 1:
            raise Exception("Amplitude of the state should sum up to 1")
        self.state[target_qubit] = initial_state
        
    def get_state(self):
        q = np.array([1])
        for key in self.state:
            q = np.kron(q,self.state[key])
        return q

def get_operator(total_qubits, gate_unitary, target_qubits, *args, **kwargs):
    # return unitary operator of size 2**n x 2**n for given gate and target qubits
    switcher_gate = {
        "x": x_gate,
        "y": y_gate,
        "z": z_gate,
        "h": hadamard_gate,
        "cx": cnot_gate,
        "u3": u3_gate
    }
    gate_func = switcher_gate.get(gate_unitary, lambda a,b: print("Invalid gate name"))
    return gate_func(total_qubits, target_qubits, args)

def set_qubit_gate(single_qubit_gate_operator, target_qubits, total_qubits):
    identity = np.identity(2)
    qubit_gates = {}
    for i in range(total_qubits):
        if i == target_qubits[0]:
            qubit_gates[i] = single_qubit_gate_operator
        else:
            qubit_gates[i] = identity
    return qubit_gates
    
def x_gate(total_qubits, target_qubits, *args): 
    X_operator = np.array([
        [0,1],
        [1,0]
    ])
    qubit_gates = set_qubit_gate(X_operator, target_qubits, total_qubits)    
    operator = np.array([1])
    for j in range(total_qubits):
        operator = np.kron(operator,qubit_gates[j])            
    return operator

def y_gate(total_qubits, target_qubits, *args):
    Y_operator = np.array([
        [0,0-1.j],
        [0+1.j,0]
    ])
    qubit_gates = set_qubit_gate(Y_operator, target_qubits, total_qubits)    
    operator = np.array([1])
    for j in range(total_qubits):
        operator = np.kron(operator,qubit_gates[j])            
    return operator  

def z_gate(total_qubits, target_qubits, *args):
    Z_operator = np.array([
        [1,0],
        [0,-1]
    ])
    qubit_gates = set_qubit_gate(Z_operator, target_qubits, total_qubits)
    operator = np.array([1])
    for j in range(total_qubits):
        operator = np.kron(operator,qubit_gates[j])
    return operator

def hadamard_gate(total_qubits, target_qubits, *args):
    h_operator = np.array([
        [1/np.sqrt(2),1/np.sqrt(2)],
        [1/np.sqrt(2),-1/np.sqrt(2)]
    ])
    qubit_gates = set_qubit_gate(h_operator, target_qubits, total_qubits)    
    operator = np.array([1])
    for j in range(total_qubits):
        operator = np.kron(operator,qubit_gates[j])    
    return operator

def u3_gate(total_qubits, target_qubits, args):
    u3_operator = np.array([
        [np.cos(args[0]['theta']/2), -1*np.exp(1.j * args[0]['lambda']) * np.sin(args[0]['theta']/2)],
        [np.exp(1.j * args[0]['phi']) * np.sin(args[0]['theta']/2), np.exp(1.j * args[0]['lambda'] + 1.j * args[0]['phi']) * np.cos(args[0]['theta']/2)]
    ])
    for i in u3_operator:
        for j in range(2):
            i[j]= complex(round(i[j].real,6),round(i[j].imag,6))
    
    qubit_gates = set_qubit_gate(u3_operator, target_qubits, total_qubits)
    operator = np.array([1])
    for k in range(total_qubits):
        operator = np.kron(operator, qubit_gates[k])
    return operator
    
def cnot_gate(total_qubits, target_qubits, args):
    P0x0 = np.array ([
    [1,0],
    [0,0]
    ])
    
    P1x1 = np.array([
    [0, 0],
    [0, 1]
    ])
    
    X_operator = np.array([
        [0,1],
        [1,0]
    ])   
    identity = np.identity(2)
    
    operator1 = P0x0
    operator2 = P1x1
    
    if target_qubits[0]<target_qubits[1]:
        qubits = [*range(target_qubits[0],target_qubits[1]+1)]
        for i in range(target_qubits[1]-target_qubits[0]-1):
            operator1 = np.kron(operator1,identity)
            operator2 = np.kron(operator2,identity)
        operator = np.kron(operator1,identity) + np.kron(operator2,X_operator)
    else:
        qubits = [*range(target_qubits[1],target_qubits[0]+1)]
        for i in range(target_qubits[0]-target_qubits[1]-1):
            operator1 = np.kron(identity,operator1)
            operator2 = np.kron(identity,operator2)
        operator = np.kron(identity,operator1) + np.kron(X_operator,operator2)
        
    added_operator = False
    final_operator = [1]
    
    for j in range(total_qubits):
        if j not in qubits:
            final_operator = np.kron(final_operator,identity)
        elif j in qubits and added_operator==False:
            final_operator = np.kron(final_operator,operator)
            added_operator = True
    
    return final_operator

def run_program(initial_state, program, *args):
    total_qubits = int(np.log2(len(initial_state)))
    for i in program:
        if i['gate']== 'u3':
            if len(args) > 0:
                i['params']['theta'] = np.around(args[0]['global_1'],decimals=6)
                i['params']['phi'] = np.around(args[0]['global_2'],decimals=6)
            matrix_operator = get_operator(total_qubits, i['gate'], i['target'],i['params'])
        else:
            matrix_operator = get_operator(total_qubits, i['gate'], i['target'])
        initial_state = np.dot(initial_state, matrix_operator)
    return np.around(initial_state, decimals= 3)

def get_counts(final_state, shots):
    prob = np.round(np.abs(final_state)**2,2)
    length_state = len(prob)
    total_qubits = int(np.log2(length_state))
    qubit_result = {}
    qubits = []
    
    for i in range(length_state):
        qubit_result[np.binary_repr(i,width=total_qubits)] = 0
        qubits.append(np.binary_repr(i,width=total_qubits))
    for j in range(shots):
        result = choice(qubits,1,p=prob)
        qubit_result[result[0]]+= 1
    predicted_output = []    
    updated_result = {}
    for key in qubit_result:
        if qubit_result[key] >0:
            predicted_output.append(qubit_result[key])
            updated_result[key] = qubit_result[key]
    return updated_result, predicted_output

def objective_function(params):
    # Generates correct params to produce bell state
    final_state =  run_program(my_qpu, my_circuit, {'global_1': params[0], 'global_2': params[1]})
    counts, predicted_output = get_counts(final_state, shots)
    actual_output = np.array([round(shots/2),round(shots/2)])
    cost = np.square(np.subtract(actual_output, predicted_output)).mean()
    return cost
    

Implementation of the following circuit
![title](images_circuit/circuit_1.png)

In [28]:
my_circuit = [
    {"gate":'h', "target":[0]},
    {"gate":'h', "target":[1]},
    {"gate":'cx', "target": [0,1]}
]

quantum_circuit = Quantum_Circuit(2)

my_qpu = quantum_circuit.get_state()

final_state = run_program(my_qpu, my_circuit)

print(final_state)

print(get_counts(final_state,1000)[0])

[0.5 0.5 0.5 0.5]
{'00': 263, '01': 279, '10': 217, '11': 241}


Implementation of cnot for non consequent qubits:
![title](images_circuit/circuit_2.png)

In [30]:
my_circuit = [
    {"gate":'h', "target":[0]},
    {"gate":'h', "target":[1]},
    {"gate":'cx', "target": [0,2]}
]

quantum_circuit = Quantum_Circuit(3)

my_qpu = quantum_circuit.get_state()

final_state = run_program(my_qpu, my_circuit)

print(final_state)

print(get_counts(final_state,1000)[0])

[0.5 0.  0.5 0.  0.  0.5 0.  0.5]
{'000': 261, '010': 255, '101': 228, '111': 256}


Implementation for initialization of state:
![title](images_circuit/circuit_3.png)

In [31]:
my_circuit = [
    {"gate":'x', "target":[0]},
    {"gate":'x', "target":[1]}
]

quantum_circuit = Quantum_Circuit(2)

initial_state = [0,1]

quantum_circuit.set_initial_state(initial_state,0)
quantum_circuit.set_initial_state(initial_state,1)

my_qpu = quantum_circuit.get_state()

final_state = run_program(my_qpu, my_circuit)

print(final_state)

print(get_counts(final_state,1000)[0])

[1. 0. 0. 0.]
{'00': 1000}


Implementation of u3 gate:
![title](images_circuit/circuit_4.png)

In [32]:
my_circuit = [
    { "gate": "u3", "params": { "theta": 3.1415, "phi": 1.15708, "lambda": -3.1415 }, "target": [0] }
]

quantum_circuit = Quantum_Circuit(1)

my_qpu = quantum_circuit.get_state()

final_state = run_program(my_qpu, my_circuit)

print(final_state)

print(get_counts(final_state,1000)[0])

[0.+0.j 1.+0.j]
{'1': 1000}


Implementation of simple function to generate correct params to produce bell state:
![title](images_circuit/circuit_5.png)

But instead of updating all three params, as I saw in the description of the question, I have updated theta and phi.

In [33]:
params = np.array([3.1415, 1.15708])

my_circuit = [
    { "gate": "u3", "params": { "theta": "global_1", "phi": "global_2", "lambda": -3.1415 }, "target": [0] },
    {"gate":'cx', "target": [0,1]}
]

shots = 1000

quantum_circuit = Quantum_Circuit(2)

my_qpu = quantum_circuit.get_state()

minimum = minimize(objective_function, params, method= 'Powell', tol=1e-6)

# prints out the correct params
print(minimum.x)

[4.66672066 2.5415622 ]


In [35]:
# check if the updated params produce bell state
params = np.array(minimum.x)

shots = 1000

my_circuit = [
    { "gate": "u3", "params": { "theta": "global_1", "phi": "global_2", "lambda": -3.1415 }, "target": [0] },
    {"gate":'cx', "target": [0,1]}
]

quantum_circuit = Quantum_Circuit(2)

my_qpu = quantum_circuit.get_state()

final_state = run_program(my_qpu, my_circuit,{ "global_1": params[0], "global_2": params[1]})

print(get_counts(final_state,shots)[0])

{'00': 504, '11': 496}


I have tried to do as much error handling as I can :) Thanks